# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f77ec6d7b80>
├── 'a' --> tensor([[-0.1513, -0.3005, -1.0893],
│                   [-1.0130, -0.3672,  0.0102]])
└── 'x' --> <FastTreeValue 0x7f77ec6d79a0>
    └── 'c' --> tensor([[-0.5801,  0.8463, -0.2808,  0.3825],
                        [ 0.1088, -0.9967, -1.6114,  1.3708],
                        [-2.0638, -2.7183,  1.5431,  0.8882]])

In [4]:
t.a

tensor([[-0.1513, -0.3005, -1.0893],
        [-1.0130, -0.3672,  0.0102]])

In [5]:
%timeit t.a

66.1 ns ± 0.0463 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f77ec6d7b80>
├── 'a' --> tensor([[ 0.3248, -1.2301, -0.7226],
│                   [-1.2188,  2.2803,  2.0032]])
└── 'x' --> <FastTreeValue 0x7f77ec6d79a0>
    └── 'c' --> tensor([[-0.5801,  0.8463, -0.2808,  0.3825],
                        [ 0.1088, -0.9967, -1.6114,  1.3708],
                        [-2.0638, -2.7183,  1.5431,  0.8882]])

In [7]:
%timeit t.a = new_value

61.9 ns ± 0.0838 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.1513, -0.3005, -1.0893],
               [-1.0130, -0.3672,  0.0102]]),
    x: Batch(
           c: tensor([[-0.5801,  0.8463, -0.2808,  0.3825],
                      [ 0.1088, -0.9967, -1.6114,  1.3708],
                      [-2.0638, -2.7183,  1.5431,  0.8882]]),
       ),
)

In [10]:
b.a

tensor([[-0.1513, -0.3005, -1.0893],
        [-1.0130, -0.3672,  0.0102]])

In [11]:
%timeit b.a

53.4 ns ± 0.0444 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-2.2425, -0.7949,  1.0973],
               [ 1.4375,  0.0683,  0.0604]]),
    x: Batch(
           c: tensor([[-0.5801,  0.8463, -0.2808,  0.3825],
                      [ 0.1088, -0.9967, -1.6114,  1.3708],
                      [-2.0638, -2.7183,  1.5431,  0.8882]]),
       ),
)

In [13]:
%timeit b.a = new_value

477 ns ± 0.124 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

865 ns ± 4.88 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 50 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

138 µs ± 627 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

139 µs ± 244 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f77424134f0>
├── 'a' --> tensor([[[-0.1513, -0.3005, -1.0893],
│                    [-1.0130, -0.3672,  0.0102]],
│           
│                   [[-0.1513, -0.3005, -1.0893],
│                    [-1.0130, -0.3672,  0.0102]],
│           
│                   [[-0.1513, -0.3005, -1.0893],
│                    [-1.0130, -0.3672,  0.0102]],
│           
│                   [[-0.1513, -0.3005, -1.0893],
│                    [-1.0130, -0.3672,  0.0102]],
│           
│                   [[-0.1513, -0.3005, -1.0893],
│                    [-1.0130, -0.3672,  0.0102]],
│           
│                   [[-0.1513, -0.3005, -1.0893],
│                    [-1.0130, -0.3672,  0.0102]],
│           
│                   [[-0.1513, -0.3005, -1.0893],
│                    [-1.0130, -0.3672,  0.0102]],
│           
│                   [[-0.1513, -0.3005, -1.0893],
│                    [-1.0130, -0.3672,  0.0102]]])
└── 'x' --> <FastTreeValue 0x7f7742413fa0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33.1 µs ± 24.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f7742413fd0>
├── 'a' --> tensor([[-0.1513, -0.3005, -1.0893],
│                   [-1.0130, -0.3672,  0.0102],
│                   [-0.1513, -0.3005, -1.0893],
│                   [-1.0130, -0.3672,  0.0102],
│                   [-0.1513, -0.3005, -1.0893],
│                   [-1.0130, -0.3672,  0.0102],
│                   [-0.1513, -0.3005, -1.0893],
│                   [-1.0130, -0.3672,  0.0102],
│                   [-0.1513, -0.3005, -1.0893],
│                   [-1.0130, -0.3672,  0.0102],
│                   [-0.1513, -0.3005, -1.0893],
│                   [-1.0130, -0.3672,  0.0102],
│                   [-0.1513, -0.3005, -1.0893],
│                   [-1.0130, -0.3672,  0.0102],
│                   [-0.1513, -0.3005, -1.0893],
│                   [-1.0130, -0.3672,  0.0102]])
└── 'x' --> <FastTreeValue 0x7f774926a970>
    └── 'c' --> tensor([[-0.5801,  0.8463, -0.2808,  0.3825],
                        [ 0.1088, -0.9967, -1.6114,  1.3708],
                 

In [23]:
%timeit t_cat(trees)

31.1 µs ± 88 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.3 µs ± 68.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.1513, -0.3005, -1.0893],
                [-1.0130, -0.3672,  0.0102]],
       
               [[-0.1513, -0.3005, -1.0893],
                [-1.0130, -0.3672,  0.0102]],
       
               [[-0.1513, -0.3005, -1.0893],
                [-1.0130, -0.3672,  0.0102]],
       
               [[-0.1513, -0.3005, -1.0893],
                [-1.0130, -0.3672,  0.0102]],
       
               [[-0.1513, -0.3005, -1.0893],
                [-1.0130, -0.3672,  0.0102]],
       
               [[-0.1513, -0.3005, -1.0893],
                [-1.0130, -0.3672,  0.0102]],
       
               [[-0.1513, -0.3005, -1.0893],
                [-1.0130, -0.3672,  0.0102]],
       
               [[-0.1513, -0.3005, -1.0893],
                [-1.0130, -0.3672,  0.0102]]]),
    x: Batch(
           c: tensor([[[-0.5801,  0.8463, -0.2808,  0.3825],
                       [ 0.1088, -0.9967, -1.6114,  1.3708],
                       [-2.0638, -2.7183,  1.5431,  0.8882]],
         

In [26]:
%timeit Batch.stack(batches)

78.8 µs ± 153 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.1513, -0.3005, -1.0893],
               [-1.0130, -0.3672,  0.0102],
               [-0.1513, -0.3005, -1.0893],
               [-1.0130, -0.3672,  0.0102],
               [-0.1513, -0.3005, -1.0893],
               [-1.0130, -0.3672,  0.0102],
               [-0.1513, -0.3005, -1.0893],
               [-1.0130, -0.3672,  0.0102],
               [-0.1513, -0.3005, -1.0893],
               [-1.0130, -0.3672,  0.0102],
               [-0.1513, -0.3005, -1.0893],
               [-1.0130, -0.3672,  0.0102],
               [-0.1513, -0.3005, -1.0893],
               [-1.0130, -0.3672,  0.0102],
               [-0.1513, -0.3005, -1.0893],
               [-1.0130, -0.3672,  0.0102]]),
    x: Batch(
           c: tensor([[-0.5801,  0.8463, -0.2808,  0.3825],
                      [ 0.1088, -0.9967, -1.6114,  1.3708],
                      [-2.0638, -2.7183,  1.5431,  0.8882],
                      [-0.5801,  0.8463, -0.2808,  0.3825],
                      [ 0.1088, -

In [28]:
%timeit Batch.cat(batches)

144 µs ± 292 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

307 µs ± 2.94 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
